In [1]:
!pip install apyori
!pip install voila
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from apyori import apriori

In [22]:
firstPassword = widgets.Password(
    value='',
    placeholder='Enter password',
    description='Password:',
    disabled=False
)
dataset = pd.read_csv('Grocery Products Purchase.csv')
transactions = []
for i in range(0,9835):
  transactions.append([str(dataset.values[i, j]) for j in range(0,32)])
rules = apriori(transactions = transactions, min_support = 0.003, min_confidence = 0.2, min_lift = 1.01, min_length = 2, max_length = 2)

In [23]:
results = list(rules)

In [24]:
def inspect(results):
  lhs = [tuple(result[2][0][0])[0] for result in results]
  rhs = [tuple(result[2][0][1])[0] for result in results]
  supports = [result[1] for result in results]
  confidences = [result[2][0][2] for result in results]
  lifts = [result[2][0][3] for result in results]
  return list(zip(lhs,rhs, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support','Confidence', 'lift'])

In [25]:
style = {'description_width': 'initial'}
limit_case = widgets.IntSlider(
    value=3,
    min=1,
    max=350,
    step=1,
    description='Show number of rows: ',
    disabled=False,
    style=style)

In [26]:
unique_columns = widgets.ToggleButtons(
    options=['lift', 'Confidence', 'Support'],
    description='Sort By:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['lift', 'confidence', 'support'],
#     icons=['check'] * 3
)


In [27]:
def update_df_length(limit, category, passcode):
    if(passcode == 'Capstone'):
        dataset = pd.read_csv('Grocery Products Purchase.csv')
        dataset = dataset.iloc[0:limit, :]
        print("Number of rows in the dataset that have been successfully loaded:"+str(len(dataset)))
        print(resultsinDataFrame.nlargest(n = limit, columns = unique_columns.value))

# Kroger Dataset Analysis 
## The password is "Capstone" 

### Adjust slider to view more rows from the database, Sort by Lift Confidence or Support

In [28]:
widgets.interactive(update_df_length, limit=limit_case, category = unique_columns, passcode = firstPassword)

interactive(children=(IntSlider(value=3, description='Show number of rows: ', max=350, min=1, style=SliderStyl…

# Adjust Data in the graph by selecting an item and a criteria
## Lift: How likely another item is purchased when selected item is purchased, a lift value > 1 represents increased likelyhood.
## Confidence: The probability a customer's cart contains the selected item and the other item.
## Support: The Popularity of selected Item + Another Item.



In [29]:
def update_df_bar(Item, Criteria, passcode):
    if(passcode == 'Capstone'):
        df = resultsinDataFrame[resultsinDataFrame['Left Hand Side'].isin(np.array(Item))].nlargest(6, np.array(Criteria)[0], keep='first')
        
        name = df['Right Hand Side'].head(6)
        price = df[np.array(Criteria)[0]].head(6)
        
        # Figure Size
        fig, ax = plt.subplots(figsize =(7, 10))
        
        # Horizontal Bar Plot
        if(np.array(Criteria)[0] == ('Support')):
            ax.barh(name, price, color = 'green')
        elif(np.array(Criteria)[0] == ('Confidence')):
            ax.barh(name, price, color = 'blue')
        else:
            ax.barh(name, price, color = 'red')
        # Remove axes splines
        for s in ['top', 'bottom', 'left', 'right']:
            ax.spines[s].set_visible(False)
        
        # Remove x, y Ticks
        ax.xaxis.set_ticks_position('none')
        ax.yaxis.set_ticks_position('none')
        
    
        
        # Add x, y gridlines
        ax.grid(b = True, color ='grey',
                linestyle ='-.', linewidth = 0.5,
                alpha = 0.2)
        
        # Show top values
        ax.invert_yaxis()

        
        # Add Plot Title
        ax.set_title(np.array(Criteria)[0] + ' between ' + np.array(Item)[0] + ' and Item',
                    loc ='left' )

        
        # Show Plot
        plt.show()

from ipywidgets import Layout
df = resultsinDataFrame
unique_item = df['Left Hand Side'].unique()
Item = widgets.SelectMultiple(
    options = unique_item.tolist(),
    value = unique_item.tolist(),
    description='Item',
    disabled=False,
    layout = Layout(width='50%', height='80px', display='flex')
)
Criteria = widgets.SelectMultiple(
    options = ['Support', 'Confidence', 'lift'],
    value = ['Support', 'Confidence', 'lift'],
    description = 'Criteria',
    disabled=False,
    layout = Layout(width='50%', height='80px',display='flex')
)

widgets.interactive(update_df_bar, Item = Item, Criteria = Criteria, passcode = firstPassword)

interactive(children=(SelectMultiple(description='Item', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, …

In [30]:
def barchart(passcode):# Plotting most popular relationships
        if(passcode == 'Capstone'):
                relationship = ['IFP, H. meat', 'flour, sugar','cheese, W. bread','liquor, beer','herbs, r. vegies','berries, w. cream','rice, r. vegies','flour, margarine','b. powder, w. cream','flour, w. cream']
                values = [11.4, 8.4, 5.9,5.2,3.9,3.7,3.7,3.6,3.6,3.2]
                fig = plt.figure(figsize = (18, 5))

                # creating the bar plot
                plt.bar(relationship, values, color ='maroon',
                        width = 0.4)

                plt.xlabel("Relationship")
                plt.ylabel("Lift")
                plt.title("Top Grocery Store Food Item Relationships")
                plt.show()
widgets.interactive(barchart, passcode = firstPassword)

interactive(children=(Password(description='Password:', placeholder='Enter password'), Output()), _dom_classes…